<a href="https://colab.research.google.com/github/roman6s/SCM_Fallstudie/blob/main/Version_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie BESS

In [15]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [16]:
! pip install -q pyscipopt

In [17]:
import pandas as pd
from pyscipopt import Model, quicksum

## Optimierungsmodell

In [18]:
scip = Model()

## Daten laden

In [19]:
folder = "WS24_Supply_Chain_Optimierung/Daten/Fallstudie"

In [20]:
# Preisprogonose
preisprognose = pd.read_excel(f"{folder}/Preisprognosen.xlsx")


CHAT

In [21]:
# Indexmenge für die 24 Stunden
T = range(1, 25)

# Parameter
cap       = 40.0     # Nominelle Kapazität [MWh]
DoD       = 0.80     # Depth of Discharge
SOC_min   = cap*(1 - DoD)   # 8 MWh
SOC_max   = cap             # 40 MWh
c_rate    = 0.5             # C-Rate
eta_round = 0.95            # Round-trip efficiency
eta_wr    = 0.985           # Wirkungsgrad Wechselrichter
cyclecost = 1500.0          # Fixe Kosten pro Zyklus
maxCycles = 2.0
p_Markt = preisprognose.groupby("Stunde")["Strompreis"].mean().tolist()

# Damit wir Lade- und Entladeverluste trennen können,
# nehmen wir z.B. sqrt(eta_round) für jede Richtung:
eta_charge    = (eta_round)**0.5 * eta_wr
eta_discharge = (eta_round)**0.5 * eta_wr

# Entscheidungsvariablen
# State of Charge je Stunde
SOC = {t: scip.addVar(
             vtype="CONTINUOUS",
             lb=SOC_min,
             ub=SOC_max,
             name=f"SOC_{t}"
          )
       for t in T}

# Lade- und Entladestrom je Stunde (jeweils in MWh, nicht negativ)
charge = {t: scip.addVar(
               vtype="CONTINUOUS",
               lb=0,
               ub=c_rate * cap,
               name=f"charge_{t}"
            )
          for t in T}

discharge = {t: scip.addVar(
                  vtype="CONTINUOUS",
                  lb=0,
                  ub=c_rate * cap,
                  name=f"discharge_{t}"
               )
             for t in T}

# Zählvariable für "verbrauchte" Zyklen
Z = scip.addVar(
      vtype="CONTINUOUS",
      lb=0,
      ub=maxCycles,
      name="cycles_used"
    )

# Nebenbedingungen

# (1) Anfangs- und End-SOC = 50% Kapazität
scip.addCons(SOC[1] == 0.5 * cap)
scip.addCons(SOC[24] == 0.5 * cap)

# (2) Ladezustandsdynamik
for t in T:
    if t == 1:
        continue
    # SOC[t] = SOC[t-1] + (eta_charge)*charge[t-1] - (1/eta_discharge)*discharge[t-1]
    scip.addCons(
        SOC[t]
        == SOC[t-1]
           + eta_charge     * charge[t-1]
           - (1/eta_discharge)* discharge[t-1]
    )

# (3) Max. 2 Zyklen über den Tag. Wir modellieren:
#     Ein "Vollzyklus" = 2*(cap * DoD). Also berechnen wir den gesamten "Throughput"
#     und setzen Z >= Throughput / [2 * (cap*DoD)].
throughput = quicksum(charge[t] + discharge[t] for t in T)
scip.addCons(
    Z >= throughput / (2.0 * cap * DoD)
)
# Z soll außerdem <= 2 sein.
scip.addCons(
    Z <= maxCycles
)

# (4) Ziel: Erlöse - Kosten pro Zyklus
#    Erlöse: p_Markt[t-1]*discharge[t]
#    Kosten: p_Markt[t-1]*charge[t]
#    Fixe Zykluskosten: cyclecost * Z
objective = quicksum(
    p_Markt[t-1] * discharge[t] - p_Markt[t-1] * charge[t]
    for t in T
) - cyclecost * Z

scip.setObjective(objective, sense="maximize")

# Modell lösen
scip.optimize()

# Ausgabe der Ergebnisse
print(f"Optimaler Zielfunktionswert: {scip.getObjVal():.2f} €")

print("\nStundenweise Lösung:")
for t in T:
    soc_val     = scip.getVal(SOC[t])
    chg_val     = scip.getVal(charge[t])
    dis_val     = scip.getVal(discharge[t])
    print(f"  t={t:2d}: "
          f"SOC={soc_val:5.2f}  "
          f"charge={chg_val:5.2f}  "
          f"discharge={dis_val:5.2f}  "
          f"Preis={p_Markt[t-1]:6.2f}")

print(f"\nEffektiv genutzte Zyklen: {scip.getVal(Z):.2f}")

Optimaler Zielfunktionswert: 3120.51 €

Stundenweise Lösung:
  t= 1: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 81.52
  t= 2: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 72.24
  t= 3: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 68.16
  t= 4: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 66.64
  t= 5: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 66.80
  t= 6: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 71.04
  t= 7: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 88.56
  t= 8: SOC=20.00  charge= 0.00  discharge=11.52  Preis= 92.68
  t= 9: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 80.92
  t=10: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 61.08
  t=11: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 43.08
  t=12: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 30.88
  t=13: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 22.44
  t=14: SOC= 8.00  charge=13.33  discharge= 0.00  Preis= 15.96
  t=15: SOC=20.80  charge=20.00  discharge= 0.00  Preis= 